# **Verlet Integration**

<i class="fa fa-home fa-2x"></i><a href="../index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/molecular-dynamics/verlet_integration.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Goals
* Understand the Verlet integration in the molecular dynamics simulations.
    * Know the relation between position, velocity, and acceleration.
    * Understand the numerical method for finite-difference.
    * Understand the order of the error of the Verlet algorithm.
    * Master the mathematics of the Verlet integration.

## Background theory

In the molecular dynamics (MD) simulations, the program needs to compute the evolution of 
the atoms' position, velocity, and acceleration. The atoms' positions contain the information 
for the structure of the molecular or material systems. Trajectory files are always written 
down from an MD simulation. Researchers can use visualization programs like 
[Visual Moleuclar Dyanmics](https://www.ks.uiuc.edu/Research/vmd/) to view the evolution 
of the systems and make scientific conclusions based on the results. Hence, it is critical 
to calculating the atoms' position, velocity, and acceleration accurately.

In the classic molecular dynamics simulations, the molecular motions are computed by 
solving the Newton equation. The numerical approximation is employed, which treats the 
velocities of the atoms as constant during a tiny period $\Delta t$. Here, we used a two-body 
system (sun and earth) to demonstrate this numerical method.

<details close>
<summary style="font-size: 20px"><b>Verlet algorithm</b></summary>
Most molecular dynamics programs employ the algorithm adopted
by Loup Verlet <a href="https://journals.aps.org/pr/abstract/10.1103/PhysRev.159.98">
[Phys. Rev. 159, 98, (1967)]</a>. There are many variants of the Verlet 
algorithm. One of the most revealing way is the so-called 
velocity Verlet algorithm, which was proposed by William C. Swope 
and Hans C. Andersen <a href="https://doi.org/10.1063/1.442716">
[J. Chem. Phys. 76, 637 (1982)]</a>. There are three equations to demonstrate 
the velocity Verlet algorithm.
    
$$\normalsize V(t+\frac{1}{2} \Delta t)=v(t)+\frac{1}{2}\Delta a(t) \quad (1)$$
$$\normalsize r(t+\Delta t)=r(t)+\Delta tv(t+\frac{1}{2}\Delta t) \quad (2)$$ 
$$\normalsize v(t+\Delta t)=v(t+\frac{1}{2}\Delta t)+\frac{1}{2}\Delta t a(t+\Delta t) \quad (3)$$
    
Eq. (1) shows that the velocities of the atoms advance half of the time
step $\Delta t$. The new positions of the atoms are updated from previous
positions with constant velocities $v(t+\frac{1}{2}\Delta t)$ as shown
in Eq. (2). The velocities are updating from the middle of the time step
$v(t+\frac{1}{2}\Delta t)$.

In the original implementation of the Verlet algorithm, the positions of
the atoms are formulated by the Taylor expansion.
    
$$\normalsize r(t+\Delta t)=r(t) + \Delta tv(t) + \frac{1}{2}\Delta t^2a(t) + \dots \quad (4)$$
$$\normalsize r(t-\Delta t)=r(t) - \Delta tv(t) + \frac{1}{2}\Delta t^2a(t) - \dots \quad (5)$$

Substitute the Eq. (4) and Eq. (5):
    
$$\normalsize r(t+\Delta t)=2r(t)-r(t-\Delta t) + \Delta t^2a(t) + O(\Delta t^4)$$
    
The velocities are not needed to compute the trajectories of the atoms.
However, the velocities are useful to estimate the kinetic energy. They
can be obtained as:
    
$$v(t) = \frac{r(t+\Delta t)-r(t-\Delta t)}{2\Delta t}$$
    
</details>

## Tasks and exercises

1. Choose a small time step, see how the earth move around the sun and check how 
the total energy and angular momentum envolute with time.

    <details>
    <summary><b>Hints</b></summary>
    The earth is moving in a ellipse orbital. The earh moves slowly in the far end
    and moves fast in the near end. However, the total energy and angular momentum
    are keeping as a constant. The earth is on the same orbital during a long time
    period.
    </details>
    
2. Choose a large time step, does the earth still keep on a same orbital? Why?

    <details>
    <summary><b>Hints</b></summary>
    When choosing a large time step, the earth is not keeping on the same 
    ellipse orbital anymore. The ellipse orbitals are shiftting for each circle.
    In the numerical method, we treat a constant velocities in the short time
    period. But this is not ture in the reality physics world. When using 
    large time step, large errors will be included into the calculations.
    </details>
    
3. In the molecular dynamics simulations, what is the problem to choose a
very short time step and a very large time step?

    <details>
    <summary><b>Hints</b></summary>
    In the molecular dynamics simulations, choosing a small time step can
    improve the accuracy of the results. However, small time step means
    taking longer computing time to obtain the results. On the other hand,
    a large time step can lead to particles too close to each others, which
    is totally wrong. Hence, choose a reasonable time step in the MD simulations
    is important.
    </details>
    
<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Interactive visualization
(be patient, it might take a few seconds to load)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from ipywidgets import Button, IntSlider, Output, VBox, HBox, Layout
from numpy.linalg import norm
import matplotlib.gridspec as gridspec

In [ ]:
def compute_force(r1, r2, m1, m2):
    """"Computer the force for the earth."""
    x = G*m1*m2/np.linalg.norm(r1-r2)**2
    v = (r2-r1)/np.linalg.norm(r2-r1)
    return x*v

In [ ]:
def compute_acceleration(f, m):
    """Compute the acceleration."""
    return f/m

In [ ]:
def verlet_update(r1, r2, v, m1, m2, dt, a):
    """Update the position, velocity and acceleration by the Verlet algorithm."""
    
    rn = 2.0*r1[-1] - r1[-2]  + dt*dt*a
    vn = (rn - r1[-2])/(2*dt)
    
    r1[-2] = r1[-1]
    r1[-1] = rn
    
    f = compute_force(rn, r2, m1, m2)
    a = compute_acceleration(f, m1)
    
    return r1, vn, a

def compute_energies(r1, r2, m1, m2, v):
    """Compute the potential energy."""
    pot = -G*m1*m2/np.linalg.norm(r1-r2)
    kint = 0.5*m1*np.linalg.norm(v)**2
    angmom = np.cross(r1-r2, m1*v)
    
    return pot, kint, angmom

In [ ]:
%matplotlib widget

m1 = 1.0
m2 = 1000.0
G = 0.2
v = [0.0, 3.0]
dt = 0.01

r1 = np.array([[10, 0],[10.0, 3.0*dt]])
r2 = np.array([0, 0])

f = compute_force(r1[-1], r2, m1, m2)
a = compute_acceleration(f, m1)

etot0 = -G*m1*m2/np.linalg.norm(r1[0]-r2) + 0.5*m1*np.linalg.norm(v)**2

output = Output()

with output:
    global fig, ax1, ax2, ax3, sun, earth, origin, avector, fvector, trace, title
    fig = plt.figure(tight_layout=True, figsize=(7.8, 5))
    fig.canvas.header_visible = False
    gs = gridspec.GridSpec(9, 5)
    
    ax1 = fig.add_subplot(gs[:, 0:3])
    ax2 = fig.add_subplot(gs[0:4, 3:5])
    ax3 = fig.add_subplot(gs[5:9, 3:5])

    sun = ax1.scatter([0.0],[0.0], s=23**2, marker="o", linewidth=0, color='gold', label = 'Sun')
    earth, = ax1.plot([10.0],[0.0], 'r.', alpha = 1.0, markersize=15, label = 'Earth')
    origin = ax1.plot([10.0],[0.0], 'kx', alpha = 1.0, markersize=15, label = 'Origin point')
    avector = ax1.quiver([10.0], [0], [0], [0], color = ['blue'], units='inches', scale = 10.0, label="a")
    fvector = ax1.quiver([10.0], [0], [0], [0], color = ['gold'], units='inches', scale = 10.0, label="v")
    trace, = ax1.plot([10.0], [0.0], "k", linewidth=0.5)
    title = ax1.set_title("")

    ax1.legend(loc=2, fontsize=10)

    ax1.set_xlim(-11.0, 11.0)
    ax1.set_ylim(-11.0, 11.0)
    ax1.grid()
    
    lpot, = ax2.plot([], [], 'k-', label='pot')
    lkint, = ax2.plot([], [], 'c-', label='kint')
    ltot, = ax2.plot([],[], 'r-', label='tot')
    ax2.axhline(etot0, color='blue', ls='--', linewidth=0.5)
    
    ax2.set_xlim(0, 20.0)
    ax2.set_ylim(-95, 60)
    ax2.legend(loc=3, ncol=3)
    ax2.set_title("Energies as a function of time", fontsize = 10)
    
    lang, = ax3.plot([],[], 'r-')
    ax3.axhline(30.0, color='blue', ls='--', linewidth=0.5)

    ax3.set_title("Angular momentum as a function of time", fontsize = 10)
    ax3.set_xlabel("Time")
    ax3.set_xlim(0, 20.0)
    ax3.set_ylim(20, 40)


xt = [];
yt = [];

lim1 = 0;
lim2 = 20.0;

t0 = 0.0;
te = [];
epot = [];
ekint = [];
etot = [];
am = [];

def init():
    return (earth, avector, fvector, trace)

    
def update(i):
    global r1, v, a, xt, yt, epot, ekint, etot, te, t0, lim1, lim2, am
    r1, v, a = verlet_update(r1, r2, v, m1, m2, dt, a)
    e1, e2, e3 = compute_energies(r1[-1,:], r2, m1, m2, v)

    earth.set_data([r1[-1, 0]], [r1[-1, 1]])
    xt.append(r1[-1, 0])
    yt.append(r1[-1, 1])
    
    epot.append(e1)
    ekint.append(e2)
    etot.append(e1+e2)
    am.append(e3)
    
    t0 = t0 + dt
    te.append(t0)
    
    if te[-1] > 20.0:
        lim1 = lim1 + dt
        lim2 = lim2 + dt
        ax2.set_xlim(lim1, lim2)
        ax3.set_xlim(lim1, lim2)

    lpot.set_data([te, epot])
    lkint.set_data([te, ekint])
    ltot.set_data([te, etot])
    lang.set_data([te, am])
    
    trace.set_data(xt, yt)
    avector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    fvector.set_offsets(np.array([r1[-1, 0], r1[-1, 1]]))
    avector.set_UVC(a[0], a[1])
    fvector.set_UVC(v[0], v[1])
    
    title.set_text("v(t) = %-5.3f" % norm(v))

    return (earth, avector, fvector, trace, lpot, lang)


anim = FuncAnimation(fig, update, frames = 20000, interval = 10,
                    init_func=init, blit=True, repeat = True)


def onClick(event):
    if button_pause.description == "Pause":
        button_pause.description = "Play"
        anim.event_source.stop()
    else:
        button_pause.description = "Pause"
        anim.event_source.start()
        

def clear_data(b):
    global xt, yt
    
    xt = [];
    yt = [];

button_pause = Button(description="Pause")
button_pause.on_click(onClick)
button_clear = Button(description="Clear trace")
button_clear.on_click(clear_data)

plt.show()

def on_dt_change(c):
    global dt, r1, xt, yt
    
    anim.event_source.stop()
    
    dt = w_dt.value*0.01
    r1 = np.array([[10, 0],[10.0, 3.0*dt]])
    
    xt = []
    yt = []
    
    anim.event_source.start()

w_dt = IntSlider(value = 1, min = 1, max = 50, step=1, description="$\Delta t$: ", orientation='horizontal',
                layout = Layout(width="400px"))
w_dt.observe(on_dt_change, names='value')

anim.pause()
display(output, w_dt, HBox([button_pause, button_clear]))

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## Legend
(How to use the interactive visualization)

### Plots
The left figure shows the sun and earth system in two dimension. The sun is
in the centre of the figure as shown in the yellow color. The earth is
represented as a red dot. The trace of the earth obital is showing
in the plot as black line. The directions of the velocity and acceleration
of the earth is showing as two vectors. The length of the vectors are
the values of the velocity and acceleration.

The top right figure shows how the energies change with time. The total energy
is a summation of the kinetic energy and potential energy. The plot is used
to monitor the energy conservation of the system.

The bottom right figure angular momentum as a function of time.

### Run the simulations
You can click the "Play & Pause" button to run or stop the simulation.
The slider is to choose the time step $\Delta t$. The "Clear trace"
button can clean the trace of the earth orbital.